In [32]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': '11',
            'day': '27'
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}


def find_mismatched_keys(data, template):
    keys_data = data.keys()
    keys_template = template.keys()
    missing_keys = ', '.join(keys_template - keys_data)
    extra_keys = ', '.join(keys_data - keys_template)
    key_errors = {
        'missing_keys': missing_keys,
        'extra_keys': extra_keys
    }
    error_type = 'mismatched keys' if (missing_keys or extra_keys) else ''
    return error_type, key_errors

def mismatched_trace_add(default_trace, key_default):
    if not default_trace:
        return default_trace
    return ', '.join(f'{key_default}.{tracer}' for tracer in default_trace.split(', '))

def mismatched_trace(key_errors, key_default):
    if key_default:
        return {
            'missing_keys': mismatched_trace_add(key_errors['missing_keys'], key_default),
            'extra_keys': mismatched_trace_add(key_errors['extra_keys'], key_default)
        }
    return key_errors


def is_same_type(data, original_type):
    try:
        return isinstance(data, original_type)
    except TypeError:
        return isinstance(data, dict) and isinstance(original_type, dict)

def find_mistyped_keys(data, template):
    mistyped_keys = set()
    for key in template:
        if not is_same_type(data[key], template[key]):
            mistyped_keys.add(key)
    error_type = 'bad types' if mistyped_keys else ''
    key_errors = ', '.join(mistyped_keys)
    return error_type, key_errors

def mistyped_trace(key_errors, key_default):
    if key_default:
        return ', '.join(f'{key_default}.{tracer}' for tracer in key_errors.split(', '))
    return key_errors


def render_error(error_type, error_trace):
    return f'{error_type} --> {error_trace}'

def validate(data, template):
    error_type = ''

    def inner(data, template, key_default):
        nonlocal error_type
        error_trace = ''

        error_type, key_errors = find_mismatched_keys(data, template)
        if error_type == 'mismatched keys':
            error_trace = mismatched_trace(key_errors, key_default)
        else:
            error_type, key_errors = find_mistyped_keys(data, template)
            if error_type == 'bad types':
                error_trace = mistyped_trace(key_errors, key_default)

        if not error_type:
            for key in data:
                if isinstance(data[key], dict):
                    inner_error_trace = inner(data[key], template[key], key)
                    if error_type:
                        if error_type == 'mismatched keys':
                            error_trace = mismatched_trace(inner_error_trace, key_default)
                        if error_type == 'bad types':
                            error_trace = mistyped_trace(inner_error_trace, key_default)
                        break

        return error_trace

    error_trace = inner(data, template, None)
    return True, '' if not error_type else render_error(error_type, error_trace)


# Play around

is_error, error_message = validate(john, template)

print(f'{is_error} {error_message}')


# String, Integer
def auto_correct_bad_types(default_object, template, error_trace):
    print('Run auto correct bad types (string, number)...')
    tracers = error_trace.split(', ')
    for tracer in tracers:
        current_data, current_template = default_object, template
        tracer_items = tracer.split('.')

        for i in range(len(tracer_items) - 1):
            current_item = tracer_items[i]
            current_data = current_data[current_item]
            current_template = current_template[current_item]

        wrong_type_property = tracer_items[-1]
        if isinstance(current_data[wrong_type_property], int) \
            or isinstance(current_data[wrong_type_property], str):
            current_data[wrong_type_property] = current_template[wrong_type_property](
                current_data[wrong_type_property])
        else:
            raise TypeError(f'Only autocorrect string and number (integer) - {tracer}')


print()
if is_error:
    error_type, error_trace = error_message.split(' --> ')
    if error_type == 'bad types':
        auto_correct_bad_types(john, template, error_trace)


print()
print(validate(john, template))


True bad types --> bio.dob.day, bio.dob.month

Run auto correct bad types (string, number)...

(True, '')
